In [1]:


from pathlib import Path
import sys

sys.path.append(str(Path().cwd().parent))
import networkx as nx
import numpy as np


In [ ]:
f = open("../graphs/graph5c.g6.txt", "r")
for line in f:
    g6 = line.strip()
    G = nx.from_graph6_bytes(g6.encode())
    A = nx.to_numpy_array(G, dtype=int)
    print(A)


D?{

DCw

DC{

DEk

DEw

DE{

DFw

DF{

DQo

DQw

DQ{

DTw

DT{

DUW

DUw

DU{

DV{

D]w

D]{

D^{

D~{



In [ ]:
# graph6 string (example)
g6 = "CF"

# read graph6
G = nx.from_graph6_bytes(g6.encode())

# convert to numpy adjacency matrix
A = nx.to_numpy_array(G, dtype=int)

print(A)